In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F


StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 4, Finished, Available)

In [3]:
%%sql
create table if not exists project_lakehouse.gold_Customer
(
  Customer_ID	string,
  Customer_Name	string,
  Segment	string,
  City	string,
  State	string,
  Country	string,
  Region	string,
  Created_TS	timestamp,
  Modified_TS	timestamp	  
)
using delta

StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 5, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
Max_Date=spark.sql('select coalesce(max(Modified_TS),"1900-01-01") from project_lakehouse.gold_Customer').first()[0]
Max_Date

StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 6, Finished, Available)

'1900-01-01'

In [6]:
#lets use our bronze table as spark dataframe
df_bronze=spark.read.table("project_lakehouse.bronze_sales")

StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 8, Finished, Available)

In [7]:

df_final = df_bronze.selectExpr("Customer_ID", "Customer_Name", "Segment", "City", "State", "Country", "Region")\
                    .where(F.col("Modified_TS")>Max_Date)\
                    .drop_duplicates() 
                    #with distinct, the create temp view did not work.

StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 9, Finished, Available)

In [8]:
df_final.createOrReplaceTempView("ViewCustomer")


StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 10, Finished, Available)

In [12]:
%%sql

merge into project_lakehouse.gold_customer as gc
using ViewCustomer as vc
on gc.Customer_ID = vc.Customer_ID
when matched then
update SET
  gc.Customer_Name = vc.Customer_ID,
  gc.Customer_ID   = vc.Customer_Name,
  gc.Segment   = vc.Segment,
  gc.City  = vc.City,
  gc.State = vc.State,
  gc.Country   = vc.Country,
  gc.Region = vc.Region,
  gc.Modified_TS   = current_timestamp()

when not matched then insert
(
    gc.Customer_ID	,
    gc.Customer_Name ,
    gc.Segment	,
    gc.City	,
    gc.State	,
    gc.Country	,
    gc.Region	,
    gc.Created_TS	,
    gc.Modified_TS
)
VALUES
(
    vc.Customer_ID	,
    vc.Customer_Name	,
    vc.Segment	,
    vc.City	,
    vc.State	,
    vc.Country	,
    vc.Region	,
    current_timestamp(),
    current_timestamp
)

StatementMeta(, 564ab781-bb54-4107-9f1a-a7f08bc158bb, 14, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>